## Define modules needed and global variables

In [10]:
""" L-galaxies python interface.

This notebook reads in halo merger graphs, performs simple operations, 
outputting the results to and HDF file.


Attributes 
----------
n_halo_max : float
    Development limiter.
debug_flag : bool
    When debugging, set to true in order to print out useful infomation.
verbosity : int
    0 prints major program steps, 1-2 are major and minor counters, 3 - Debugging diags.
file_parameters : str
    Filepath to input parameters yml file.
display_parameters : bool
    Whether or not to display the contents of the yml file.
parameters : '(dict of str: (dict of str: str))'
    Input parameters from yml file. Contains input/output filepaths, constants etc.
f_baryon : float
    Baryon fraction for halos.
io_nRec : int
    Size of HDF5 io buffer.
dtype_halo : dtype
    Numpy dtype describing the types of the different class attributes.
    
"""


import numpy as np
import h5py as h5
import yaml


n_halo_max = np.inf


debug_flag = False


verbosity = 1 

file_parameters='Input_Params/input_params.yml'

display_parameters = True

parameters = yaml.load(open(file_parameters),Loader=yaml.Loader)

if display_parameters:
    for item in parameters:
        print("{:20s}: {}".format(item,parameters[item]))

f_baryon=parameters['cosmology']['fBaryon']['Value']
io_nRec=parameters['performance']['io_nRec']['Value']


dtype_halo=np.dtype([
    ('graph_ID',np.int32),
    ('snap_ID',np.int32),
    ('halo_ID',np.int32),
    ('catalog_ID',np.int64),
    ('mass',np.float32),
    ('mass_baryon',np.float32),
    ('mass_from_progenitors',np.float32)
])

inputFiles          : {'graphFile': '/lustre/scratch/astro/ab898/FYP/Input Data/mega_graph_mini.hdf5'}
outputFiles         : {'haloFile': '/lustre/scratch/astro/ab898/FYP/Output Data/SMT13HaloOutput.hdf5', 'galaxyFile': '/lustre/scratch/astro/ab898/FYP/Output Data/SMT13GalaxyOutput.hdf5'}
cosmology           : {'OmegaM': {'Description': 'Matter density parameter', 'Value': 0.3, 'Units': 'None'}, 'fBaryon': {'Description': 'Baryon fraction', 'Value': 0.155, 'Units': 'None'}}
modelSwitches       : {'HOD': {'Description': 'Halo occupation description for stars (as primitive test)', 'Value': True}}
performance         : {'io_nRec': {'Description': 'Size of HDF5 io buffer (number of records)', 'Value': 1000}}


##  Halo Properties class

In [2]:
class HaloProperties:
    """A container for the properties needed for each halo.
    
    No sophisticated methods, just the constructor. 

    Parameters
    ----------
    graph_ID : str
        The graph number in reference to the HDF group.
    snap_ID : str
        The snapshot number in reference to the HDF dataset.
    halo_ID : str
        The halo's ID number within the HDF dataset.

    Attributes
    ----------
    graph_ID : str
        The graph number in reference to the HDF group.
    snap_ID : str
        The snapshot number in reference to the HDF dataset.
    halo_ID : str
        The halo's ID number within the HDF dataset.
    done : bool
        Whether or not the halo has been processed.
    catalog_ID : int
        The ID of the halo corresponding to the original catalog, not just this file.
    mass : int
        Root mass of the halo taken from attributes of the graph.
    mass_baryon : float
        Mass of Baryons within the halo.
    mass_from_progenitors : float 
        Total mass of all the progenitor halos.
    mass_baryon_from_progenitors : float
        Total mass of all the Baryons contained within the progenitor halos.
    desc_start : int
        The start index of the descendent halos for each halo.

    """   
    def __init__(self,graph_ID,snap_ID,halo_ID):
        
        self.graph_ID=graph_ID
        self.snap_ID=snap_ID
        self.halo_ID=halo_ID
        self.done=False
        self.catalog_ID=-1
        self.mass=0.
        self.mass_baryon=0.
        self.mass_from_progenitors=0.
        self.mass_baryon_from_progenitors=0.
        self.desc_start = 0
        if parameters['modelSwitches']['HOD']: 
            self.mass_stars=0.
            self.mass_stars_from_progenitors=0.

## High level driver routines

In [6]:
def open_graph_input():
    """Opens input graph file using module level parameters.
    
    Returns
    -------
    :obj:`File`
        HDF5 file open in read mode
    
    """
    
    return h5.File(parameters['inputFiles']['graphFile'],'r')

def open_galaxy_output():
    """ Opens the output HDF5 file in write mode.
    
    Returns
    -------
    :obj: 'File'
        HDF5 file open in write mode.
    
    """
    return h5.File(parameters['outputFiles']['galaxyFile'],'w')

def close_graph_io(open_HDF5_file):
    """Closes an open HDF5 file.
    
    Parameters
    ----------
    open_HDF5_file : :obj: 'File'
        An open HDF5 file.
    
    Returns
    -------
    None
    
    """
    open_HDF5_file.close()
    return None

def open_halo_output():
    """ Open halo output file, create iobuffer, and output empty dataset.
    
    Returns
    -------
    :obj: 'File'
        An HDF5 file open in write mode.
    ndarray
        Empyy numpy array of varying type. See module level variables dtype_halo. 
    :obj: 'Dataset'
        An HDF5 dataset.
    int
        Amount of halos that have been outputted. 
    
    """
    halo_output_file = h5.File(parameters['outputFiles']['haloFile'],'w')
    halo_output_data = np.empty(io_nRec,dtype=dtype_halo)
    halo_output_dataset = halo_output_file.create_dataset('Halos',(0,),maxshape=(None,),
                                                    dtype=dtype_halo,compression='gzip')
    halo_output_iRec = 0
    return halo_output_file,halo_output_data,halo_output_dataset,halo_output_iRec  




def output_halos(halos,halo_output_data,halo_output_dataset,halo_output_iRec):
    """Constructs a structured numpy array and calls flush_output module
    
    Parameters
    ----------
    halo_output_file : :obj: 'File'
        An open HDF5 file for the data to be written to.
    halo_output_data : ndarry
        Structured numpy array to store halo attributes.
    halo_output_dataset : :obj: 'Dataset'
        HDF5 dataset to which the data should be stored.
    halo_output_iRec: int
        Amount of halos that have been outputted.
    
    Returns
    -------
    int
        Amount of halos that have been outputted.
    
    """
    
    for halo in halos:
        halo_output_data[halo_output_iRec]['graph_ID']=halo.graph_ID
        halo_output_data[halo_output_iRec]['snap_ID']=halo.snap_ID
        halo_output_data[halo_output_iRec]['halo_ID']=halo.halo_ID
        halo_output_data[halo_output_iRec]['catalog_ID']=halo.catalog_ID
        halo_output_data[halo_output_iRec]['mass']=halo.mass
        halo_output_data[halo_output_iRec]['mass_baryon']=halo.mass_baryon
        halo_output_data[halo_output_iRec]['mass_from_progenitors']=halo.mass_from_progenitors
        halo_output_iRec+=1
        if halo_output_iRec==io_nRec: halo_output_iRec=flush_output(halo_output_iRec,
                                                                    halo_output_data,halo_output_dataset)
            
            
    return halo_output_iRec

    

def flush_output(n_rec,output_data,output_dataset):
    """Writes data to an HDF5 dataset
    
    Parameters
    ----------
    n_rec : int
        Amount of halos that have been outputted.
    output_data : ndarry
        Structured numpy array containing halo properties.
    ouput_dataset : :obj: 'Dataset'
        HDF5 dataset in which the data should be stored.
    
    Returns
    -------
    int
        Amount of halos that have been outputted
        
    """
    output_dataset.resize((output_dataset.shape[0]+n_rec,))
    output_dataset[-n_rec:]=output_data[:n_rec]
    n_rec=0
    return n_rec

In [11]:
def process_halo(halo):
    """Small function that checks halo-properties before calling calculations.
    
    Controls what calculations are done. E.g. progenitors are not gathered
    if halo is in the first generation. 
    
    
    Parameters
    ----------
    halo : :obj: 'Class'
        HaloProperties class object
    
    Returns
    -------
    None
        Halo class object is updated
        
    """
    if verbosity>=3: print('Processing halo ',halo.halo_ID)
    if halo.done==True: 
        print('Warning: processHalo: halo ',str(halo),' already processed.')
        assert False
    read_properties(halo)
    calc_mass_to_desc(halo)
    # Omit gatherProgenitors for first generation of halos!
    if halo_properties_last_snap != None: gather_progenitors(halo)
    set_baryon_fraction(halo)
    # fixStellarFraction(halo) # Dummy routine.
    halo.done=True
    
    return None
    
def read_properties(halo):
    """Reads the mass and descendent start index from HDF5 group attributes
    
    Parameters
    ----------
    halo : :obj: 'Class'
        HaloProperties class object
    
    Returns
    -------
    None
        Halo class object is updated
        
    """
    # Reads halo properties from the input graph file
    #halo.catalogID=graphInputFile[halo.graphID][halo.snapID][halo.haloID].attrs.get('halo_catalog_halo_ids')
    halo.mass = graph_input_file[halo.graph_ID].attrs.get('root_mass') #halo_mass was here
    halo.desc_start = graph_input_file[halo.graph_ID]['desc_start_index'][halo.halo_ID]
    
    return None

def calc_mass_to_desc(halo):
    """Calculates the mass that does into each descendant in proportion to the desc_mass contribution
    
    Parameters
    ----------
    halo : :obj: 'Class'
        HaloProperties class object
    
    Returns
    -------
    None
        Halo class object is updated
        
    """
    # Determines how much mass goes to each descendant, in proportion to desc_mass_contribution
    desc_mass_contribution=graph_input_file[halo.graph_ID]['direct_desc_contribution'][halo.desc_start:]
    desc_mass=np.array(desc_mass_contribution/np.sum(desc_mass_contribution)*halo.mass,dtype=np.float64)
    halo.desc_mass=dict(zip(graph_input_file[halo.graph_ID]['direct_desc_ids'][halo.desc_start:],desc_mass))

    return None


def set_baryon_fraction(halo):
    """ Caclulates the mass of baryons in the halo
    
    Uses the global f_baryon (baryon fraction) variable to calculate the total
    mass provided by baryons.
    
    Parameters
    ----------
    halo : :obj: 'Class'
        HaloProperties class object
    
    Returns
    -------
    None
        Halo class object is updated
        
    """
    halo.mass_baryon=f_baryon*max(halo.mass,halo.mass_from_progenitors)
    return None
    
def gather_progenitors(halo):
    """ Gathers the progenitors and calculates the mass from each one.
    
    I'm not entirely sure on the inner mechanics of this function. I will 
    probably re-write in trying to understand better.
    
    Parameters
    ----------
    halo : :obj: 'Class'
        HaloProperties class object
    
    Returns
    -------
    None
        Halo class object is updated
    
    """
    # Collects information about material inherited from progenitors
    prog0_halo_ID=int(halo_properties_last_snap[0].halo_ID)
    print(prog0_halo_ID)
    for prog_halo_ID in graph_input_file[halo.graph_ID]['direct_prog_ids']:
        
        
        # Position in progenitor (last_snap) halo list
        if debug_flag and verbosity>=3 : print('direct_prog_ids =',prog_halo_ID)
            
    
        
        prog_index_last_snap=int(prog_halo_ID)-prog0_halo_ID
        
        #print(prog_haloID,halo.haloID,prog0_haloID,prog_index_lastSnap) 
        #print(halo.desc_mass)
        if debug_flag and verbosity>=3: print('prog_index_last_snap =',prog_index_last_snap)
        # Check halo association
        # This is needed because HDF5 may store halos in a different order - 
        # if this happens, will need to add code to do a search over all halos
        # in lastSnap, or to rearrange into ascending order.
        if debug_flag:
            print(halo_properties_last_snap[prog_index_last_snap].halo_ID)
            print(prog_halo_ID)
            assert int(halo_properties_last_snap[prog_index_last_snap].halo_ID) == int(prog_halo_ID)
        # Now gather the appropriate information from the progenitor halo
        halo.mass_from_progenitors+=halo_properties_last_snap[prog_index_last_snap].desc_mass[int(halo.halo_ID)]

    return None



## Main routine

In [14]:
# Open graph input file
# Note: can't pass undefined argument into function
graph_input_file=open_graph_input()

# Open output files
# Note: can't pass undefined argument into function
halo_output_file,halo_output_data,halo_output_dataset,halo_output_iRec  = open_halo_output()
# Note: no attempt here to include sub-halos (galaxies).  Let's get halos right first!
#openGalaxyOutput(galaxyOutputFile)

# Iteratively loop over halos, doing whatever processing is required.
# This assumes that halos properties depend only upon those halos in their immediate past in the merger graph.

# Loop over MergerGraphs.
n_halo=0
#nHaloGraph=0
#nHaloSnap=0

amount_of_graphs = len(graph_input_file['/nhalos_in_graph/'][:])

for graph_ID in range(0,amount_of_graphs):
    
    if verbosity >= 2: print('Processing graph',graph_ID)
    graph=graph_input_file[str(graph_ID)]
    halo_properties_last_snap = None
    snap_ID_old=-1
    
    for snap_ID in graph['snapshots']:
        
    
        
        if verbosity>=2: print('        snapshot',snap_ID)
            
        # Initialise halo properties
        halo_properties_this_snap = [HaloProperties(str(graph_ID),snap_ID,halo_ID) for 
                                    halo_ID in graph['/{}/graph_halo_ids'.format(graph_ID)][:]]
         
        #print(vars(haloProperties_thisSnap[1]))
        
        print('Graph ID: {}, Snap ID: {}'.format(graph_ID,snap_ID))
        for halo in halo_properties_this_snap: 
            print('Halo ID: {}'.format(halo.halo_ID))
            process_halo(halo)
            n_halo +=1
            if verbosity>=1 and n_halo%1000==0: print('Processed {:d} halos'.format(n_halo))
        
        
        halo_output_iRec=output_halos(halo_properties_this_snap,halo_output_data,
                                      halo_output_dataset,halo_output_iRec)
        
        
        
        
        
        del halo_properties_last_snap
        #gc.collect() # garbage collection -- safe but very slow.
        halo_properties_last_snap = halo_properties_this_snap
        # Delete reference to this memory to free variable for new use.
        del halo_properties_this_snap
        # Temporary halt to limit to finite time
        if n_halo>=n_halo_max:
            
            if halo_output_iRec>0: halo_output_iRec=flush_output(halo_output_iRec,halo_output_data,
                                                                 halo_output_dataset)

            close_graph_io(halo_output_file)
            
            assert False
        


if not debug_flag: del halo_properties_last_snap

# Close input file
close_graph_input(graph_input_file)

# Close output files

if halo_output_iRec>0: halo_output_iRec=flush_output(halo_output_iRec,halo_output_data,halo_output_dataset)
    
close_graph_io(halo_output_file)


    

Graph ID: 0, Snap ID: 30
Halo ID: 0
Halo ID: 1
Halo ID: 2
Halo ID: 3
Halo ID: 4
Halo ID: 5
Halo ID: 6
Halo ID: 7
Halo ID: 8
Halo ID: 9
Halo ID: 10
Halo ID: 11
Halo ID: 12
Halo ID: 13
Halo ID: 14
Halo ID: 15
Halo ID: 16
Halo ID: 17
Halo ID: 18
Halo ID: 19
Halo ID: 20
Halo ID: 21
Halo ID: 22
Halo ID: 23
Halo ID: 24
Halo ID: 25
Halo ID: 26
Halo ID: 27
Halo ID: 28
Halo ID: 29
Halo ID: 30
Halo ID: 31
Halo ID: 32
Halo ID: 33
Halo ID: 34
Halo ID: 35
Halo ID: 36
Halo ID: 37
Halo ID: 38
Halo ID: 39
Halo ID: 40
Halo ID: 41
Halo ID: 42
Halo ID: 43
Halo ID: 44
Halo ID: 45
Halo ID: 46
Halo ID: 47
Halo ID: 48
Halo ID: 49
Halo ID: 50
Halo ID: 51
Halo ID: 52
Halo ID: 53
Halo ID: 54
Halo ID: 55
Halo ID: 56
Halo ID: 57
Halo ID: 58
Halo ID: 59
Halo ID: 60
Graph ID: 0, Snap ID: 31
Halo ID: 0
0


KeyError: 0

In [13]:
close_graph_io(halo_output_file)
halo_output_file.close()